In [1]:
# libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime


In [ ]:
# reading dataset

df = pd.read_csv('/home/pedro/Desktop/autoprognosis/code_v6/INFLUD-21-09-2020.csv',sep=';',encoding = "ISO-8859-1")

# Hospitalized people with age small than 110:

df = df[df['PCR_SARS2']==1][df['HOSPITAL']==1][df['NU_IDADE_N']<=110][df['EVOLUCAO'] != 3][df['EVOLUCAO'] != 9][df['EVOLUCAO'].notnull()]
print(df.shape)

/home/pedro/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15,60,62,63,64,92,94,108,117,121,123) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# Pre-processing time variable

df['time'] = pd.to_datetime(df['DT_ENCERRA'], format='%d/%m/%Y',errors ='coerce').subtract(pd.to_datetime(df['DT_SIN_PRI'], format='%d/%m/%Y', errors ='coerce'))
df['time'] = df['time']/ np.timedelta64(1, 'D')


In [ ]:
# To identifying public and private hospital

df_public = pd.read_csv('/home/pedro/Desktop/autoprognosis/Archive-SUS/SUS-SIM.txt', sep='\t')
df_private = pd.read_csv('/home/pedro/Desktop/autoprognosis/Archive-SUS/SUS-NAO.txt', sep='\t')

public  =  set(df_public.iloc[:,0])
#print(len(public))

private  =  set(df_private.iloc[:,0])
#print(len(private))


df.loc[df['CO_UNI_NOT'].isin(public) == True, 'HEALTH_SYSTEM'] = 1
df.loc[df['CO_UNI_NOT'].isin(private) == True, 'HEALTH_SYSTEM'] = 0

print(df['HEALTH_SYSTEM'].value_counts())
print(df['HEALTH_SYSTEM'].shape)

print(df['CO_UNI_NOT'].isnull().sum())


In [ ]:
# Selecting people from North with COVID-19
df_N = df[df['PCR_SARS2']==1][df['HOSPITAL']==1]\
[(df['SG_UF_NOT']=='RO')|(df['SG_UF_NOT']=='TO')|(df['SG_UF_NOT']=='AM')|(df['SG_UF_NOT']=='RR')\
|(df['SG_UF_NOT']=='AC')|(df['SG_UF_NOT']=='PA')|(df['SG_UF_NOT']=='AP')]


# Selecting people from Northeast with COVID-19
df_NE = df[df['PCR_SARS2']==1][df['HOSPITAL']==1]\
[(df['SG_UF_NOT']=='RN')|(df['SG_UF_NOT']=='PE')|(df['SG_UF_NOT']=='SE')| (df['SG_UF_NOT']=='CE')\
|(df['SG_UF_NOT']=='PB')| (df['SG_UF_NOT']=='BA')|(df['SG_UF_NOT']=='AL')| (df['SG_UF_NOT']=='PI')\
|(df['SG_UF_NOT']=='MA')]


# Selecting people from Center-West with COVID-19
df_CO = df[df['PCR_SARS2']==1][df['HOSPITAL']==1]\
[(df['SG_UF_NOT']=='DF')|(df['SG_UF_NOT']=='MS')|(df['SG_UF_NOT']=='GO')| (df['SG_UF_NOT']=='MT')]


# Selecting people from Southest with COVID-19
df_SE = df[df['PCR_SARS2']==1][df['HOSPITAL']==1]\
[(df['SG_UF_NOT']=='SP') | (df['SG_UF_NOT']=='RJ')|(df['SG_UF_NOT']=='ES')|(df['SG_UF_NOT']=='MG')]


# Selecting people from South with COVID-19
df_S = df[df['PCR_SARS2']==1][df['HOSPITAL']==1]\
[(df['SG_UF_NOT']=='RS')|(df['SG_UF_NOT']=='SC')|(df['SG_UF_NOT']=='PR')]


print('confirmed data in North :',df_N.shape[0])
print('confirmed data in Northeast:',df_NE.shape[0])
print('confirmed data in Center-West:',df_CO.shape[0])
print('confirmed data in Southest:',df_SE.shape[0])
print('confirmed data in South:',df_S.shape[0])

In [ ]:

labels = ['North', 'Northwest', 'Midwest', 'Southwest', 'South']


fig = plt.figure(1, figsize=(9, 6))

# Create an axes instance
ax = fig.add_subplot(111)


all_data = [df_N['time'].dropna(), df_NE['time'].dropna(),\
df_CO['time'].dropna(),df_SE['time'].dropna(),\
df_S['time'].dropna()]



# Create the boxplot
bp = ax.boxplot(all_data , showfliers=False, labels=labels, sym='k+')


ax.set_xticklabels(labels=labels,fontsize=19)
plt.yticks(fontsize=16)
plt.setp(bp['fliers'], markersize=3.0)
plt.ylabel('Time',fontsize=20)
plt.savefig('boxplot_time.pdf')
plt.show()
plt.close()